## This only exists for testing new data functions I may add, please make sure you are importing `datatools.jl` when working on the assignment 

In [2]:
using CSV, DataFrames, Statistics

In [3]:
const ROOT_PATH = @__DIR__
const DATA_PATH = "$ROOT_PATH/data"

"/home/justin/Documents/Spring2024/NumericalAnalysis/NA-FinalProject/data"

In [4]:
"""
get_datafiles(foldername::String)

takes in the name of a folder or path to subfolder in /data/
and returns an array of the paths to the different file names 


data_left_sideOf_center_dr = get_datafiles("Center/Left")

--> ["NA-FinalProject/data/Center/Left/Location-Justin-04-04", etc]
"""
function get_datafiles(foldername::String)
     filelist = readdir("$DATA_PATH/$foldername", join=true)
     return filelist
end


get_datafiles

In [5]:
"""
get_folder_dataframes(foldername::String)

takes in the name of a folder or the path to a subfolder in /data/
returns a vector of all the dataframes in that folder



sweetwater_dataframes = get_folder_dataframes("Sweetwater")

"""
function get_folder_dataframes(foldername::String)
     filelist = get_datafiles(foldername)

     dfs = []
     for path in filelist
          data = CSV.File(open(path)) |> DataFrame 
          push!(dfs, data)
     end
     return dfs
end

get_folder_dataframes

In [6]:
"""
filter_matching_rows(dfs::Vector{Any}, column_name::String; round_to::Int=6)

takes in a vector of dataframes and a column to filter them by. It looks through the dataframes
and keeps rows in which the values match. Can specify how close you want values to be by 
using round_to to adjust the precision. Returns a vector of the resulting dataframes


filtered_dfs = (dfs, "latitude"; round_to=5)
"""
function filter_matching_rows(dfs::Vector{Any}, column_name::String="latitude"; round_to::Int=5)
    # Ensure there is at least one data frame
    if isempty(dfs)
        error("The input vector must contain at least one data frame.")
    end
    
    # Ensure the column exists in all data frames
    for df in dfs
        if !(column_name in names(df))
            error("The specified column must exist in all data frames.")
        end
    end
    
    # Apply rounding and find the intersection of rounded values across all data frames
    rounded_values_sets = [Set(round.(df[!, column_name], digits=round_to)) for df in dfs]
    matching_values = reduce(intersect, rounded_values_sets)
    
    # Filter all data frames to only include rows with values (rounded) in the matching set
    filtered_dfs = [
        filter(row -> round(row[column_name], digits=round_to) in matching_values, df)
        for df in dfs
    ]
    
    return filtered_dfs
end


filter_matching_rows

In [7]:
"""
get_filtered_points(dfs::Vector{Any}, column_name::String; round_to::Int=6, only_unique::Bool=true)

Takes in vector of dataframes generated by get_folder_dataframes. User specifies a column_name 
by which to compare the dataframes. Values are rounded 6 by default, but can be changed. 
The dataframes are trimmed based on these matching rounded values, and only the columns 
latitude, longitude, and altitude are returned. These are all that are needed for displaying the
points 


filtered_points = get_filtered_points(folder_dataframes, "latitude"; round_to=3)

"""
function get_filtered_points(dfs::Vector{Any}, column_name::String="latitude"; round_to::Int=5, only_unique::Bool=true)
     filtered_dfs = filter_matching_rows(dfs, column_name; round_to=round_to)

     filtered_points = []

     for df in filtered_dfs
          map!(x->round(x, digits=round_to), df[!, column_name], df[!, column_name])
          new_df = select(df, ["latitude", "longitude", "altitude"])
          if only_unique
               unique!(new_df, column_name)
          end
          sort!(new_df, column_name)
          push!(filtered_points, new_df)
     end
     
     return filtered_points
end

get_filtered_points

In [8]:
"""
stack_df_vectors(dfs::Vector{Any})

Takes in a vector of dataframes and combines them into one dataframe. It adds in a column titled "sample" to indicate which
data sample a given row comes from. 
"""
function stack_df_vectors(dfs::Vector{Any})
     for i = 1:length(dfs)
          df = dfs[i]
          df.sample = fill("Sample $i", nrow(df))
     end

     a = dfs[1]
     for i = 2:length(dfs)
          append!(a, dfs[i])
     end
     
     return a
end

stack_df_vectors

In [9]:
"""
combine_df_vectors(dfs::Vector{Any})

This function takes in a vector of dataframes from the same road (similar samples).
It will compare the vectors and average them together. Unique sample latitudes will instead 
be appended into the new dataframe. The function assumes that you are passing in the returned value of 
get_filtered_points, but allows for you to set already_filtered=false if the data is unfiltered
"""
function combine_df_vectors(dfs::Vector{Any}; already_filtered::Bool=true, round_to::Int=5)

     if already_filtered
          dfs = dfs
     else
          dfs = get_filtered_points(dfs; round_to=round_to)
     end

     points = Dict()

     for df in dfs
          for i = 1:length(df[!, :1])
               if df.latitude[i] in keys(points)
                    push!((points[df.latitude[i]]).alt, df.altitude[i])
               else
                    points[df.latitude[i]] = (alt=[df.altitude[i]], long=df.longitude[i])
               end
          end
     end 

     lats = [k for k in keys(points)]
     longs = [v.long for v in values(points)]
     alts = [mean(v.alt) for v in values(points)]

     d = DataFrame(latitude=lats, longitude=longs, altitude=alts)

     sort!(d, "latitude")

     return d
          
end

combine_df_vectors

In [10]:
"""
    get_spaced_nodes(x, y, n=10; rev=true)

Takens in x and y values. returns 'n' number of spaced out points from the data.
By default this will return evenly spaced points, but setting fluctuation=SOME_INT will 
cause each index to fluctuate by ± the inputted value
"""
function get_spaced_nodes(x, y, n=10; fluctuation::Int=0)
     len = length(x)
     indices = round.(Int, LinRange(1, len, n))


     for i = 2:length(indices)-1
        indices[i] += rand((-1 * fluctuation):fluctuation)
     end
    
     @show indices

     if !issorted(x)
          sort!(x)
     end
           
     xs = x[indices]
     ys = y[indices]

     return xs, ys
 end

get_spaced_nodes

In [11]:
"""
    get_all_dataframes()

Gets all the dataframes from the folders and stores them in a dictionary. The keys of the 
dictionary are the street that the dataframes belong to. The values of the dictionary are
vectors of dataframes, with each dataframe representing a sample taken on said street.
"""
 function get_all_dataframes()
     folders = ["Center/East", "Center/West", "Gale/East", "Gale/West", "Museum", "Sweetwater"]

     all_dfs = Dict()

     for folder in folders
          all_dfs[folder] = get_folder_dataframes(folder)
     end

     return all_dfs

end

get_all_dataframes

Everything below this line is in progress

---

In [12]:
function stack_all_streets(dict::Dict{Any, Any})
     combined_df = DataFrame()
 
     for (street, dfs) in dict
         street_df = stack_df_vectors(dfs)
         
         street_df.street = fill(street, nrow(street_df))
         
         if isempty(combined_df)
             combined_df = street_df
         else
             append!(combined_df, street_df)
         end
     end
     
     return combined_df
 end


stack_all_streets (generic function with 1 method)

In [37]:
all_dfs = get_all_dataframes()

df = stack_all_streets(all_dfs);

smallest = df.altitude[argmin(df.altitude)]

df.altitude = (df.altitude .- smallest .+ 0.001) .* 2 

11055-element Vector{Float64}:
  19.45004165537194
  19.45004165537194
  14.45004165537194
  14.45004165537194
  14.45004165537194
  14.45004165537194
  17.10987915229881
  17.10987915229881
  17.30955540703777
  17.30955540703777
   ⋮
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349
 172.450038794349

In [38]:
CSV.write("alldata.csv", df)

"alldata.csv"

11055-element Vector{Float64}:
 -3.0999999046325684
 -3.0999999046325684
 -3.5999999046325684
 -3.5999999046325684
 -3.5999999046325684
 -3.5999999046325684
 -3.334016154939882
 -3.334016154939882
 -3.314048529465986
 -3.314048529465986
  ⋮
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137
 12.199999809265137